<a href="https://colab.research.google.com/github/yutao-data/Financial-Calculator/blob/main/investment_daily_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tushare
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.8.0
    Uninstalling websocket-client-1.8.0:
      Successfully uninstalled websocket-client-1.8.0
Mounted at /content/drive


In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts
import os
from datetime import datetime, timedelta
import time

# Initialize the Tushare API
api_key_path = '/content/drive/My Drive/tushare_data/api_key.txt'
with open(api_key_path, 'r') as file:
    api_key = file.readline().strip()
pro = ts.pro_api(api_key)

# 股票市场

In [28]:
drive_path = '/content/drive/My Drive/tushare_data/status/stock_status'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

In [97]:
today = datetime.today().strftime('%Y%m%d')

def get_index_pe(index_code):
    index_data = pro.index_dailybasic(ts_code=index_code, trade_date=today, fields='ts_code,trade_date,pe')
    if not index_data.empty:
        pe = index_data.iloc[0]['pe']
        return pe
    return None

# 获取上证指数的市盈率
sh_pe = get_index_pe('000001.SH')
sh_pe = f"{sh_pe:.2f}" if sh_pe is not None else "暂无数据"

# 获取创业板指的市盈率
cyb_pe = get_index_pe('399006.SZ')
cyb_pe = f"{cyb_pe:.2f}" if cyb_pe is not None else "暂无数据"

# 获取两市成交数据
stock_data = pro.daily(trade_date=today)
stock_data_path = os.path.join(drive_path, f'stock_data_{today}.csv')
stock_data.to_csv(stock_data_path, index=False)

total_volume = stock_data['vol'].sum() / 1e8  # 转换为亿股
total_turnover = stock_data['amount'].sum() / 1e5  # 转换为亿元
rise_count = (stock_data['pct_chg'] > 0).sum()  # 上涨家数
fall_count = (stock_data['pct_chg'] < 0).sum()  # 下跌家数

# 获取沪深港通资金流向数据
moneyflow_hsgt = pro.moneyflow_hsgt(trade_date=today)
moneyflow_hsgt_path = os.path.join(drive_path, f'moneyflow_hsgt_{today}.csv')
moneyflow_hsgt.to_csv(moneyflow_hsgt_path, index=False)

northbound_inflow = moneyflow_hsgt['north_money'].sum() / 1e2  # 转换为亿元
southbound_inflow = moneyflow_hsgt['south_money'].sum() / 1e2  # 转换为亿元

daily_basic = pro.daily_basic(trade_date=today)
daily_basic_path = os.path.join(drive_path, f'daily_basic_{today}.csv')
daily_basic.to_csv(daily_basic_path, index=False)

total_market_cap = daily_basic['total_mv'].sum() / 1e4  # 转换为亿元

print(f"今日两市成交额：{total_turnover:.2f} 亿元, 上涨：{rise_count}家, 下跌：{fall_count}家, 北向资金净流入：{northbound_inflow:.2f} 亿元, 南向资金净流入：{southbound_inflow:.2f} 亿元, 上证指数市盈率：{sh_pe}，创业板指市盈率：{cyb_pe}。")

今日两市成交额：5896.99 亿元, 上涨：2341家, 下跌：2741家, 北向资金净流入：-49.85 亿元, 南向资金净流入：7.45 亿元, 上证指数市盈率：12.77，创业板指市盈率：25.58。


# 债券市场

In [30]:
drive_path = '/content/drive/My Drive/tushare_data/status/bond_status'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

In [61]:
today = datetime.today().strftime('%Y%m%d')
start_date = (datetime.today() - timedelta(days=5)).strftime('%Y%m%d')

# 获取国债收益率曲线数据
bond_yield_data = pro.yc_cb(ts_code='1001.CB', curve_type='0', start_date=start_date, end_date=today)
bond_yield_data = bond_yield_data.sort_values(by='trade_date', ascending=False)
bond_yield_data_10y = bond_yield_data[bond_yield_data['curve_term'] == 10]

if len(bond_yield_data_10y) > 1:
    bond_yield_today = bond_yield_data_10y.iloc[0]
    bond_yield_yesterday = bond_yield_data_10y.iloc[1]
    latest_bond_yield_today = bond_yield_today['yield']
    bond_yield_date_today = bond_yield_today['trade_date']
    latest_bond_yield_yesterday = bond_yield_yesterday['yield']
    bond_yield_date_yesterday = bond_yield_yesterday['trade_date']
    bond_yield_change = latest_bond_yield_today - latest_bond_yield_yesterday
else:
    latest_bond_yield_today = bond_yield_data_10y.iloc[0]['yield']
    bond_yield_date_today = bond_yield_data_10y.iloc[0]['trade_date']
    latest_bond_yield_yesterday = "暂无数据"
    bond_yield_date_yesterday = "暂无数据"
    bond_yield_change = "暂无数据"

# 获取最近的SHIBOR利率数据
shibor_data = pro.shibor(start_date=start_date, end_date=today)
shibor_data = shibor_data.sort_values(by='date', ascending=False)

if len(shibor_data) > 1:
    shibor_today = shibor_data.iloc[0]
    shibor_yesterday = shibor_data.iloc[1]
    latest_shibor_today = shibor_today['on']
    shibor_date_today = shibor_today['date']
    latest_shibor_yesterday = shibor_yesterday['on']
    shibor_date_yesterday = shibor_yesterday['date']
    shibor_change = latest_shibor_today - latest_shibor_yesterday
else:
    latest_shibor_today = shibor_data.iloc[0]['on']
    shibor_date_today = shibor_data.iloc[0]['date']
    latest_shibor_yesterday = "暂无数据"
    shibor_date_yesterday = "暂无数据"
    shibor_change = "暂无数据"

print(f"今日国债收益率（10年期）[{bond_yield_date_today}]：{latest_bond_yield_today}%，相较前一日（{bond_yield_date_yesterday}）变化：{bond_yield_change:.2f}%。")
print(f"最新上海银行间同业拆放利率（{shibor_date_today}）：{latest_shibor_today}%，相较昨日（{shibor_date_yesterday}）变化：{shibor_change:.2f}%。")


今日国债收益率（10年期）[20240729]：2.1574%，相较前一日（20240726）变化：-0.04%。
最新上海银行间同业拆放利率（20240726）：1.689%，相较昨日（20240725）变化：-0.07%。


# 美元兑人民币汇率

In [71]:
# 获取最近的外汇汇率数据
fx_rate_data = pro.fx_daily(ts_code='USDCNH.FXCM', start_date=start_date, end_date=today)
fx_rate_data = fx_rate_data.sort_values(by='trade_date', ascending=False)

if len(fx_rate_data) > 1:
    fx_rate_today = fx_rate_data.iloc[0]
    fx_rate_yesterday = fx_rate_data.iloc[1]
    latest_fx_rate_today = fx_rate_today['bid_close']
    fx_rate_date_today = fx_rate_today['trade_date']
    latest_fx_rate_yesterday = fx_rate_yesterday['bid_close']
    fx_rate_date_yesterday = fx_rate_yesterday['trade_date']
    fx_rate_change = latest_fx_rate_today - latest_fx_rate_yesterday
elif len(fx_rate_data) == 1:
    fx_rate_today = fx_rate_data.iloc[0]
    latest_fx_rate_today = fx_rate_today['bid_close']
    fx_rate_date_today = fx_rate_today['trade_date']
    latest_fx_rate_yesterday = "暂无数据"
    fx_rate_date_yesterday = "暂无数据"
    fx_rate_change = "暂无数据"
else:
    latest_fx_rate_today = "暂无数据"
    fx_rate_date_today = "暂无数据"
    latest_fx_rate_yesterday = "暂无数据"
    fx_rate_date_yesterday = "暂无数据"
    fx_rate_change = "暂无数据"

print(f"美元兑人民币汇率（{fx_rate_date_today}）：{latest_fx_rate_today}，相较昨日（{fx_rate_date_yesterday}）变化：{fx_rate_change*100:.3f}%。")

美元兑人民币汇率（20240728）：7.26177，相较昨日（20240727）变化：0.047%。


# Financial News

In [93]:
drive_path = '/content/drive/My Drive/tushare_data/news'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

print("Fetching latest news from Sina...")
news_data = pro.news(src='sina', start_date='2024-07-21 09:00:00', end_date='2024-07-29 10:10:00')

print("Columns in news data:\n", news_data.columns)

random_news_data = news_data.sample(n=5)

print("Randomly selected news data sample:\n", random_news_data)

news_list = []

for index, row in random_news_data.iterrows():
    datetime_str = row.get('datetime', "暂无数据")
    title = row.get('title', "暂无数据") if pd.notna(row['title']) else "暂无数据"
    content = row.get('content', "暂无数据") if pd.notna(row['content']) else "暂无数据"

    news_desc = f"""
    时间：{datetime_str}
    标题：{title}
    内容：{content}
    """
    news_list.append(news_desc.strip())

print("\n最新的五个新闻：")
for news in news_list:
    print(news)
    print('-' * 80)

Fetching latest news from Sina...
Columns in news data:
 Index(['datetime', 'content', 'title'], dtype='object')
Randomly selected news data sample:
                 datetime                                            content  \
885  2024-07-27 17:10:16  【李斌公布第二代蔚来NIO Phone售价：6499元起售】2024蔚来创新科技日上，蔚来董...   
916  2024-07-27 15:53:46  【第七届进博会倒计时100天 签约展览面积超36万平方米】第七届中国国际进口博览会将于今年1...   
227  2024-07-29 02:56:40  【稳增长加码信号明确 A股市场信心有望逐渐修复】上周，A股市场未能延续前期反弹趋势，主要指数...   
654  2024-07-28 06:44:02  【又一突破！华能石岛湾核电站扩建一期工程将开工】今天上午，我国先进核电基地——华能石岛湾核电...   
672  2024-07-28 05:04:21  特朗普2024年比特币大会演讲要点总结：\n1、参会原因是想让美国在数字加密货币保持领先地位...   

    title  
885        
916        
227        
654        
672        

最新的五个新闻：
时间：2024-07-27 17:10:16
    标题：
    内容：【李斌公布第二代蔚来NIO Phone售价：6499元起售】2024蔚来创新科技日上，蔚来董事长李斌公布第二代蔚来NIO Phone售价，与前代产品保持一致，仍为6499元起售，蔚来APP、京东、抖音即刻开售。
--------------------------------------------------------------------------------
时间：2024-07-27 15:53:46
    标题：
    内容：【第七届进博会倒计时100天 签约展览面积超36万平方米】第